In [ ]:
from dataclasses import dataclass
import time
from tqdm import tqdm
from multiprocessing import Pool
from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec

import obspy as op
from obspy import read,read_inventory, UTCDateTime, Stream, Trace
from obspy.clients.fdsn.client import Client
from obspy.signal.rotate import rotate_ne_rt
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees

import json
import glob
import os
import numpy as np
from itertools import combinations
import pandas as pd
from scipy.signal import spectrogram, detrend, resample,savgol_filter,decimate,hilbert

import pyarrow.feather as feather

import datetime

from sklearn.linear_model import LinearRegression,HuberRegressor,TheilSenRegressor

In [ ]:
# ==================
# Configuration file
# ==================

# Folders input

EVENT_DIR = '/medata01/SEISCOMP_DATA/'

XML_DIR = '/home/sysop/dados_posdoc/PROJETO_RSIS/XML/'

# Folders output

ORIENTATION_OUTPUT = '/home/sysop/dados_posdoc/PROJETO_RSIS/ORIENTATION_OUTPUT/'

# -------------------------------

# Stations and OBSs information

STATION_LST = ['DUB01','MAN01','CAM01','VAS01']

PERIOD_BANDS2 = [0.03,0.05]


# -------------------------------

# create figures?

VERBOSE = True

# Input parameters

FIRSTDAY = '2017-01-01'
LASTDAY = '2017-12-31'

# default parameters to define the signal and noise windows used to estimate the SNR:
# - the signal window is defined according to time after Rayleigh-wave arrival:

# Rayleigh-wave time windows start
TIME_START_P_REGIONAL = 40

# Rayleigh-wave time windows final
TIME_FINAL_P_REGIONAL = 800

# Returns pairs and spectral SNR array whose spectral SNRs are all >= minspectSNR
minspectSNR = 2

#RESAMPLING
NEW_SAMPLING_RATE = 1

# -------------------------------
# Mappoing parameters

LLCRNRLON_LARGE = -50
URCRNRLON_LARGE = -38
LLCRNRLAT_LARGE = -30
URCRNRLAT_LARGE = -12

# -------------------------------

# Constants and parameters

ONEDAY = datetime.timedelta(days=1)

# -------------------------------

# MULTIPROCESSING

num_processes = 20

# =================
# Filtering by date
# =================

fday = UTCDateTime(FIRSTDAY)
lday = UTCDateTime(LASTDAY)
INTERVAL_PERIOD = [UTCDateTime(x.astype(str)) for x in np.arange(fday.datetime,lday.datetime+ONEDAY,ONEDAY)]
INTERVAL_PERIOD_DATE = [str(x.year)+'.'+"%03d" % x.julday for x in INTERVAL_PERIOD]

In [ ]:
# =========
# Functions
# =========

# --------------------------------------------------------------------------
def chael_selby_algorithm(tr1,tr2,trZ,noise,ang,dphi):
            
    # Rotate through and find max
    
    Szr = np.zeros(len(ang))
    Rzr = np.zeros(len(ang))
    SNR = np.zeros(len(ang))
                            
    for k, a in enumerate(ang):
        R, T = rotate_ne_rt(tr1, tr2, a)
                                
        # Hilbert transformed radial trace data
        trR_H = np.imag(hilbert(R))
                
        Czr = np.corrcoef(trZ,trR_H)[0,1]
        Czz = np.corrcoef(trZ,trZ)[0,1]
        Crr = np.corrcoef(trR_H,trR_H)[0,1]
    
        Szr[k] = Czr/Czz
        
        Rzr[k] = Czr/np.sqrt(np.dot(Czz,Crr))
    
        SNR[k] = np.abs(R).max()/noise
                            
    return Szr,Rzr,SNR

#-------------------------------------------------------------------------------

In [ ]:
# ============
# Main program
# ============
print('==================')
print('Downloading events')
print('==================')
print('\n')

start_time = time.time()

client = Client('IRIS')

cat = client.get_events(starttime=fday, endtime=lday, minmagnitude=6.5)
print(cat)

In [ ]:
print('\n')
print('============================')
print('Calculating the orientation:')
print('============================')
print('\n')

SNR_MIN = 10
CC_MIN = 0.4

for sta in tqdm(STATION_LST,total=len(STATION_LST),desc='Stations'):
    
    # ---------------------------
    # Retrieving events waveforms
    # ---------------------------
    
    sta_orientation = []
    sta_Szr_max = []
    sta_SRN = []
    
    for event in tqdm(cat, total=len(cat),desc=sta+' orientation'):
        # --------------------
        # Check if file exists
        event_time = event.origins[-1].time
        event_name = event.origins[-1].time.strftime('%Y.%j.%H.%M.%S')

        year = event.origins[-1].time.strftime('%Y')
        julian_day = event.origins[-1].time.strftime('%j')

        network = 'ON'
        station = sta      
        
        output_FEATHER_FILES_ORIENTATION = ORIENTATION_OUTPUT+'FEATHER_FILES/'+sta+'/'
        
        file_feather_name = output_FEATHER_FILES_ORIENTATION+event_name+'_ORIENTATION_data.feather'

        station_pwd = glob.glob(EVENT_DIR+year+'/'+network+'/'+station+'/*')

        if os.path.isfile(file_feather_name):
            pass
    
        else:
            
            if (len([i for i in station_pwd if 'HHE.D' in i or 'HH1.D' in i]) > 0 and
                len([i for i in station_pwd if 'HHN.D' in i or 'HH2.D' in i]) > 0 and
                len([i for i in station_pwd if 'HHZ.D' in i]) > 0):

                if (len(glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                    len(glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                    len(glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)) > 0):
                
                    #try:
                    
                        file_HHE = glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                        file_HHN = glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                        file_HHZ = glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)[0]
    
                        # --------
                        # Data HHE
                        
                        tr2_data_file = op.read(file_HHE)
                        tr2_data_file.trim(event_time+TIME_START_P_REGIONAL,event_time+TIME_FINAL_P_REGIONAL)
                        tr2_data_file.decimate(factor=10, strict_length=False)
                        tr2_data_file.taper(type='cosine',max_percentage=0.1)
                        tr2_data_file.filter('bandpass',freqmin=PERIOD_BANDS2[0],freqmax=PERIOD_BANDS2[1],zerophase=True)
                        tr2_data_filtered = tr2_data_file[0].data
            
                        # --------
                        # Data HHN
            
                        tr1_data_file = op.read(file_HHN)
                        tr1_data_file.trim(event_time+TIME_START_P_REGIONAL,event_time+TIME_FINAL_P_REGIONAL)
                        tr1_data_file.decimate(factor=10, strict_length=False)
                        tr1_data_file.taper(type='cosine',max_percentage=0.1)
                        tr1_data_file.filter('bandpass',freqmin=PERIOD_BANDS2[0],freqmax=PERIOD_BANDS2[1],zerophase=True)
                        tr1_data_filtered = tr1_data_file[0].data
                        
                        # --------
                        # Data HHZ
                        
                        trZ_data_file = op.read(file_HHZ)
                        trZ_data_file.trim(event_time+TIME_START_P_REGIONAL,event_time+TIME_FINAL_P_REGIONAL)
                        trZ_data_file.decimate(factor=10, strict_length=False)
                        trZ_data_file.taper(type='cosine',max_percentage=0.1)
                        trZ_data_file.filter('bandpass',freqmin=PERIOD_BANDS2[0],freqmax=PERIOD_BANDS2[1],zerophase=True)
                        trZ_data_filtered = trZ_data_file[0].data
                        trZ_time = trZ_data_file[0].times()
            
                        # ---------------
                        # Import XML file
                        
                        xml_file = glob.glob(XML_DIR+network+'.'+sta+'*')[0]
                        station_xml = op.read_inventory(xml_file)
                            
                        # Epicentral distance:
            
                        sta_lon = station_xml[-1][-1][-1].longitude
                        sta_lat = station_xml[-1][-1][-1].latitude
            
                        ev_lat = event.origins[-1].latitude
                        ev_lon = event.origins[-1].longitude
            
                        dist_pair,az_pair,baz_pair = gps2dist_azimuth(ev_lat, ev_lon,sta_lat, sta_lon)
                        gcarc_pair = kilometers2degrees(dist_pair/1000)
                        gcarc_pair_round = round(gcarc_pair)
                        dist_pair_round = round(dist_pair)
                        
                        #-------------------------------------------------------------------------------------------------------------------------------
                        # Calculating Hilbert transform of vertical trace data
                        
                        trZ_H_data_filtered = np.imag(hilbert(trZ_data_filtered))
                                
                        #-------------------------------------------------------------------------------------------------------------------------------
                        
                        signal_window = (trZ_time >= TIME_START_P_REGIONAL) & (trZ_time <= TIME_FINAL_P_REGIONAL)
                        noise_window = (trZ_time < TIME_START_P_REGIONAL) != (trZ_time > TIME_FINAL_P_REGIONAL)
                        
                        noise = tr1_data_filtered[noise_window].std()

                        tr2 = tr2_data_filtered[signal_window]
                        tr1 = tr1_data_filtered[signal_window]
                        trZ = trZ_data_filtered[signal_window]
                        
                        dphi = 0.1
                        ang = np.arange(0., 360., dphi)
                        
                        Szr,Rzr,SNR = chael_selby_algorithm(tr1,tr2,trZ,noise,ang,dphi)
            
                        # Get argument of maximum of Czr:
                        best_index = np.argmax(Szr)                     
                        
                        # Get azimuth and correct for angles above 360
                        phi = baz_pair - ang[best_index]
                        phi = phi % 360
                            
                        if phi > 180:
                            phi = 360 - phi 
                        
                        # Get argument of maximum coherence:
                        
                        Szr_max = Szr[best_index]
                        Rzr_max = Rzr[best_index]
                        SNR_max = SNR[best_index]
            
                        sta_orientation.append(phi)
                        sta_Szr_max.append(Szr_max)
                        sta_SRN.append(SNR_max)
            
                        # ----------------------------------------------------------------------------------------------------
                        # Creating a Pandas DataFrame:
                        column_info = [sta,dist_pair,gcarc_pair,baz_pair,Szr_max,Rzr_max,SNR_max,phi]
            
                        columns_header = ['station','distance','gcarc','baz','Szr_max','Rzr_max','SNR_max','orient']
            
                        orient_rayleigh_df = pd.DataFrame(column_info, index=columns_header).T
            
                        # ----------------------------------------------------------------------------------------------------
                        # Convert from pandas to Arrow and saving in feather formart file
                        os.makedirs(output_FEATHER_FILES_ORIENTATION,exist_ok=True)
                        feather.write_feather(orient_rayleigh_df, file_feather_name)
                        # ----------------------------------------------------------------------------------------------------
                        
                        if VERBOSE == True:
            
                            # --------------------
                            # fig CrossCorrelation
                            fig = plt.figure(figsize=(20, 10))
                            fig.suptitle('Evento: '+event_name+'\n Δ: '+str(gcarc_pair_round)+'° | M: '+str(event.magnitudes[-1].mag)+' '+event.magnitudes[-1].magnitude_type,fontsize=20)
            
                            gs = gridspec.GridSpec(3, 2,wspace=0.2, hspace=0.5)
            
                            new_R, new_T = rotate_ne_rt(tr1_data_filtered, tr2_data_filtered, phi)
            
                            ax1 = fig.add_subplot(gs[0,0])
                            ax1.plot(trZ_time,new_T,'-k')
                            ax1.set_ylabel('HHE')
                            ax1.set_xlabel('Timelag (s)')
                            ax1.axvline(x=TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                            ax1.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
            
                            ax2 = fig.add_subplot(gs[1,0], sharey=ax1, sharex=ax1)
                            ax2.plot(trZ_time,new_R,'-k')
                            ax2.plot(trZ_time,trZ_H_data_filtered,'--r')
                            ax2.set_ylabel('HHN')
                            ax2.set_xlabel('Timelag (s)')
                            ax2.axvline(x=TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                            ax2.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
            
                            ax3 = fig.add_subplot(gs[2,0], sharey=ax1, sharex=ax1)
                            ax3.plot(trZ_time,trZ_data_filtered,'-k')
                            ax3.set_ylabel('HHZ')
                            ax3.set_xlabel('Timelag (s)')
                            ax3.axvline(x=TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                            ax3.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
            
                            ax4 = fig.add_subplot(gs[0,1])
                            ax4.plot(ang,Rzr,'.k')
                            ax4.plot(phi,Rzr_max,'*r')
                            ax4.set_ylabel('$R_{rz}$')
                            ax4.set_xlabel('Orientation Angle (deg)')
            
                            ax5 = fig.add_subplot(gs[1,1], sharex=ax4)
                            ax5.plot(ang,Szr,'.k')
                            ax5.plot(phi,Szr_max,'*r')
                            ax5.set_ylabel('$S_{rz}$')
                            ax5.set_xlabel('Orientation Angle (deg)')
            
                            ax6 = fig.add_subplot(gs[2,1])
                            ax6.plot(ang,SNR,'.k')
                            ax6.plot(phi,SNR_max,'*r')
                            ax6.set_ylabel('SNR')
                            ax6.set_xlabel('Orientation Angle (deg)')
            
                            if (Rzr_max >= CC_MIN) & (SNR_max >= SNR_MIN):
                                label = 'good'
                            else:
                                label = 'bad'
                            output_figure_ORIENTATION = ORIENTATION_OUTPUT+'ORIENTATION_FIGURES/EARTHQUAKES/'+sta+'/'
                            os.makedirs(output_figure_ORIENTATION,exist_ok=True)
                            fig.savefig(output_figure_ORIENTATION+'ORIENTATION_'+event_name+'_'+label+'.png',dpi=300)
                            plt.close()
        
                    #except:
                        #pass
    #Creating the figure
    fig = plt.figure(figsize=(10, 10))
    fig.suptitle(sta,fontsize=20)

    gs = gridspec.GridSpec(1, 1,wspace=0.2, hspace=0.5)

    ax1 = fig.add_subplot(gs[0])

    sta_orientation_np = np.array(sta_orientation)
    sta_cc1_max_np = np.array(sta_Szr_max)
    sta_SRN_np = np.array(sta_SRN)

    th_mask = (sta_cc1_max_np >= CC_MIN) & (sta_SRN_np >= SNR_MIN)
    th_mask_ = (sta_cc1_max_np < CC_MIN) & (sta_SRN_np < SNR_MIN)

    sta_orientation_np_good = sta_orientation_np[th_mask]
    sta_cc1_max_np_good = sta_cc1_max_np[th_mask]

    sta_orientation_np_bad = sta_orientation_np[th_mask_]
    sta_cc1_max_np_bad = sta_cc1_max_np[th_mask_]

    #Simple Linear Regression With scikit-learn
    #Provide data:
    x = np.array(range(len(sta_cc1_max_np_good))).reshape((-1, 1))
    y = sta_orientation_np_good

    x_pred = np.arange(start=0,stop=1,step=0.1)

    model = TheilSenRegressor(n_jobs=12,max_iter=500)
    model.fit(x, y)
    y_pred = model.predict(np.array(range(len(x_pred))).reshape((-1, 1)))

    ax1.plot(sta_orientation_np_bad,sta_cc1_max_np_bad,'.k')
    ax1.plot(sta_orientation_np_good,sta_cc1_max_np_good,'ok')
    ax1.plot(np.mean(y_pred),np.mean(sta_cc1_max_np_good),'*r')

    ax1.set_ylim(0,1)
    ax1.set_xlim(0,360)
    ax1.set_ylabel('$R_{rz}$')
    ax1.set_xlabel('Orientation (degrees)')

    output_figure_ORIENTATION = ORIENTATION_OUTPUT+'ORIENTATION_FIGURES/EARTHQUAKES/'+sta+'/'
    os.makedirs(output_figure_ORIENTATION,exist_ok=True)
    fig.savefig(output_figure_ORIENTATION+'ORIENTATION_TOTAL_'+sta+'.png',dpi=300)
    plt.close()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')
